In [1]:
import warnings
warnings.filterwarnings("ignore")
import re
import os
import datetime as dt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv

In [2]:
generalData_df      =  pd.read_csv("C:/Users/raja/general_data.csv")
employee_survey_df  =  pd.read_csv("C:/Users/raja/employee_survey_data.csv")
manager_survey_df   =  pd.read_csv("C:/Users/raja/manager_survey_data.csv")
intime_df           =  pd.read_csv("C:/Users/raja/in_time.csv")
outtime_df          =  pd.read_csv("C:/Users/raja/out_time.csv")

In [ ]:
df1=pd.merge(generalData_df,employee_survey_df,on='EmployeeID')
final_df=pd.merge(df1,manager_survey_df,on='EmployeeID')

if(len(final_df.columns.tolist())   == (len(generalData_df.columns.tolist()) + len(employee_survey_df.columns.tolist()) + len(manager_survey_df.columns.tolist())) - 2):
  print("OK")

In [ ]:
final_df.head()

In [ ]:
print(final_df.isnull().sum().sum())
final_df.dropna(inplace=True)
print(final_df.isnull().any().sum())

Checking for duplicates

In [ ]:
final_df_dupliactedRows=final_df[final_df.duplicated( keep=False)].shape[0]
if(final_df_dupliactedRows==0):
  print("there is No duplicate elements in Final_df")
else:
  print("there is  duplicate elements in Final_df")
  

# EDA

We will cover how to visually analyze: <br>
Numerical variables with histograms,<br>
Categorical variables with count plots,<br>
Relationships between numerical variables with scatter plots, joint plots, and pair plots, and<br>
Relationships between numerical and categorical variables with box-and-whisker plots and complex conditional plots.<br>

In [ ]:
#For all columns doing countplots
fig = plt.figure(figsize=(26,30))
for idx,i in enumerate(final_df.columns.tolist()):
    ax=plt.subplot(8,4,idx+1)
    sns.countplot(x=i,data=final_df,ax=ax)
plt.show()

In [ ]:
#Divide into NumericalColumns
columns=final_df.columns.tolist()
num_col_eda=['Age','DistanceFromHome','PercentSalaryHike','MonthlyIncome','TotalWorkingYears','YearsAtCompany']
final_df[num_col_eda].hist(figsize=(10,10))
plt.show()

Output ::

1. Key Observation from Above Plot are
   -Except Age most of the Columns are in Skew Distribistion form
   -Age Feature Distribution is almost Normal Distribution
2. As logistic regression does not require independent variables to be normal distributed .so i am not changing distribution of features which are skewed into the normal Distribution

In [ ]:
num_attrition=num_col_eda+['Attrition']
education=['Below-College','College','Bachelor','Master','Doctor']
environmentsatisfaction=['Low','Medium','High','Very High']
jobinvolvement=['Low','Medium','High','Very High']
jobsatisfaction=['Low','Medium','High','Very High']
relationshipsatisfaction=['Low','Medium','High','Very High']
performancerating=['Low','Good','Excellent','Outstanding']
worklifebalance=['Bad','Good','Better','Best']

val=[education,environmentsatisfaction,jobinvolvement,jobsatisfaction,performancerating,worklifebalance]
cat1=['Education','EnvironmentSatisfaction','JobInvolvement','JobSatisfaction','PerformanceRating','WorkLifeBalance']

cat_col_eda=set(columns) - set(num_col_eda)
cat_col_eda=set(cat_col_eda) - set(cat1)
fig = plt.figure(figsize=(16,20))

In [ ]:
#2. chaning nums as x_ticks to   categorys as x_ticks
fig = plt.figure(figsize=(30,28))
for idx,i in enumerate(zip(cat1,val)):
    #crosstab = pd.crosstab(index=final_df[i[0]], columns=final_df["Attrition"])
    ax=plt.subplot(6,4,idx+1)
    #crosstab.plot(kind="bar",stacked=True,ax=ax)
    #sns.countplot(x=i[0],data=final_df,ax=ax)
    sns.countplot(final_df[i[0]],hue=final_df['Attrition'],ax=ax)
    ax.set(xticks=range(len(i[1])), xticklabels=[j for j in i[1]])
plt.show()

In [ ]:
#Divide into CategoricalColumns
#1. with categorys as x_ticks
fig = plt.figure(figsize=(20,26))
for idx,i in enumerate(cat_col_eda):
    crosstab = pd.crosstab(index=final_df[i], columns=final_df["Attrition"])
    ax=plt.subplot(6,4,idx+1)
    crosstab.plot(kind="bar",stacked=True,ax=ax)
    #sns.countplot(x=i[0],data=final_df,ax=ax)
    #ax.set(xticks=range(len(i[1])), xticklabels=[j for j in i[1]])
plt.show()

In [ ]:
 #2. chaning nums as x_ticks to   categorys as x_ticks
fig = plt.figure(figsize=(20,26))
for idx,i in enumerate(zip(cat1,val)):
    crosstab = pd.crosstab(index=final_df[i[0]], columns=final_df["Attrition"])
    ax=plt.subplot(6,4,idx+1)
    crosstab.plot(kind="bar",stacked=True,ax=ax)
    #sns.countplot(x=i[0],data=final_df,ax=ax)
    ax.set(xticks=range(len(i[1])), xticklabels=[j for j in i[1]])
plt.show()

In [ ]:
#for i in num_attrition:
fig = plt.figure(figsize=(20,26))
for idx,i in enumerate(num_attrition):
  crosstab = pd.crosstab(index=final_df[i], columns=final_df["Attrition"])
  ax=plt.subplot(6,4,idx+1)
    #sns.boxplot(x=i,data=final_df,ax=ax)
  
  crosstab.plot(kind="bar",stacked=True,ax=ax)

# OUTLIER Detection

In [ ]:
#Box Plot for finding "Outiler" in our data
fig = plt.figure(figsize=(20,26))
for idx,i in enumerate(num_col_eda):
    ax=plt.subplot(6,4,idx+1)
    sns.boxplot(x=i,data=final_df,ax=ax)

Results from Above Graph:
from above BoxPlot<br> Trying to find is there any outliers in Numerical columns <br>
can Observe outliers on MonthlyIncome , TotalWorkingYears and YearsAtCompany Columns<br>
from observing on that columns can say those columns some of values not outliers Because there is highly possibilites on<br> occuring those numerical values on those features or columns<br>

Feature Selection

In [ ]:
final_df.head()

In [ ]:
final_df['age_fs1']=final_df['Age'].map(lambda x: "20-40" if(x<40) else "40-60")

In [ ]:
sns.countplot(final_df["age_fs1"],hue=final_df['Attrition'])
plt.show()

# Finding Coorelation
## Corelation on Numerical Features

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))         # Sample figsize in inches

sns.heatmap(final_df.corr(),cmap='YlOrRd',linewidths=.5,ax=ax)

### There is is no higher dependency between column to columns from above heatMap Plot
Categorical Varibles Coorelation ::
ChiSquare Test for Independence ::
Hypothesis Testing Conditions ::
Our hypotheses will be:

Null Hypothesis (H0)

H0 :: There is no relationship between 2 categorival varibles ie .. Both features or varibles are independent of each other
Alternate Hypothesis (H1)

H1 :: There is Relationship between 2 categorical varibles .ie .. Both features or varibles are independent of each other

In [ ]:
categorical=[i for i in final_df.columns.tolist() if(final_df[i].dtype==object)]
categorical.remove("Over18")

import scipy.stats
from scipy.stats import chi2
results=[]
lst=[]
#final=[]
input_features=[]
chisqr_result=[]
for i in categorical:
  #print("*"*6 + i + "*"*6)
    final=[]
    for j in categorical:
    #print("*"*6 + j + "*"*6)
    #print("*"*6 + i +"--" + j + "*"*6)
    #Contingency Table
        contingency_table=pd.crosstab(final_df[i],final_df[j])
    #print('contingency_table :-\n',contingency_table)
    
    #Observed Values
        Observed_Values = contingency_table.values
    #print("Observed Values :-\n",Observed_Values)
    
    #Expected Values
    #import scipy.stats
        b=scipy.stats.chi2_contingency(contingency_table)
        Expected_Values = b[3]
    #print("Expected Values :-\n",Expected_Values)


    #Degree of Freedom
        no_of_rows=Observed_Values.shape[0]
        no_of_columns=Observed_Values.shape[1]
        df=(no_of_rows-1)*(no_of_columns-1)
    #print("Degree of Freedom:-",df)

    #Significance Level 5%
        alpha=0.05


    #chi-square statistic - χ2
    #from scipy.stats import chi2
        chi_square=sum([(o-e)**2./e for o,e in zip(Observed_Values,Expected_Values)])
        chi_square_statistic=chi_square[0]+chi_square[1]
    #print("chi-square statistic:-",chi_square_statistic)


    #critical_value
        critical_value=chi2.ppf(q=1-alpha,df=df)
    #print('critical_value:',critical_value)

    #p-value
        p_value=1-chi2.cdf(x=chi_square_statistic,df=df)
    #print('p-value:',p_value)


    #print('Significance level: ',alpha)
    #print('Degree of Freedom: ',df)
    #print('chi-square statistic:',chi_square_statistic)
    #print('critical_value:',critical_value)
    #print('p-value:',p_value)
    
    #lst1=[df,chi_square_statistic,critical_value,p_value]
        lst1=[df,chi_square_statistic,p_value]
    
    #compare chi_square_statistic with critical_value and p-value which is the probability of getting chi-square>0.09 (chi_square_statistic)
        if(chi_square_statistic>=critical_value):
      #print("Reject H0,There is a relationship between 2 categorical variables")
          test_stat=1
        else:
      #print("Retain H0,There is no relationship between 2 categorical variables")
          test_stat=0
    
        if(p_value<=alpha):
        ##print("Reject H0,There is a relationship between 2 categorical variables")
            p_val=1
        else:
        #print("Retain H0,There is no relationship between 2 categorical variables")
            p_val=0
        
        if((test_stat==1) and (p_val==1) ):
              final_output=1
        else:
              final_output=0
    
        lst.append(lst1)
        final.append(final_output)
        input_features.append((i,j))
    
    
    results.append(lst)
    chisqr_result.append(final)

print(input_features) 
print(results)
print(chisqr_result)

In [ ]:
# plot the heatmap
sns.heatmap(chisqr_result, 
        xticklabels=categorical,
        yticklabels=categorical,annot=True)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split


attrition_lable=final_df['Attrition']
final_df=final_df.drop('Attrition',1)

attrition_lable=attrition_lable.map(lambda x : 1 if(x=='Yes') else 0)


X_train, X_test, y_train, y_test = train_test_split(final_df,attrition_lable,test_size=0.20,stratify=attrition_lable, random_state=42)

In [ ]:
#foe i in [X_train,X_cv,X_test]:
X_train=pd.get_dummies(X_train)
X_test=pd.get_dummies(X_test)
  

In [ ]:
from sklearn.tree import DecisionTreeClassifier

ml = DecisionTreeClassifier()
ml.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
y_pred = ml.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
precision_score(y_test, y_pred)

In [ ]:
recall_score(y_test, y_pred)

In [ ]:
f1_score(y_test, y_pred)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
cr = classification_report(y_test, y_pred)
print(cr)

# Notes:

1) The performance of developed model is excellent and it can be deployed to predict future employee attrition.

2) We found a lot of insights about employee attrition during EDA.

3) Few major caused of employee attrition includes human resource work, bad work life balance, frequent travels and unmarried employees. So, in order to reduce attrition company should focus on these reasons.

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
model = XGBClassifier()

In [ ]:
# fit the model with the training data
model.fit(X_train, y_train)

In [ ]:
predict_test = model.predict(X_test)
predict_test

In [ ]:
testaccuracy = accuracy_score(y_test, predict_test)
print('accuracy_score on test dataset : ', testaccuracy)

In [ ]:
# Check for the VIF values of the feature variables. 
from statsmodels.stats.outliers_influence import variance_inflation_factor
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif = pd.DataFrame()
vif['Features'] = X_train.columns
vif['VIF'] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif.tail()

In [ ]:
from sklearn import metrics
# Confusion matrix 
confusion = metrics.confusion_matrix(y_test, predict_test )
print(confusion)

In [ ]:
TP = confusion[1,1] # true positive 
TN = confusion[0,0] # true negatives
FP = confusion[0,1] # false positives
FN = confusion[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our model
testsensitivity= TP / float(TP+FN)
testsensitivity

In [ ]:
# Let us calculate specificity
testspecificity= TN / float(TN+FP)
testspecificity

In [ ]:
# Calculate false postive rate - predicting Attrition when customer does not have Attrited
print(FP/ float(TN+FP))

In [ ]:
# Positive predictive value 
print (TP / float(TP+FP))

In [ ]:
from sklearn.metrics import precision_score, recall_score
precision_score(y_test, predict_test)

In [ ]:
recall_score(y_test, predict_test)

In [ ]:
print("Test Data Accuracy     :{} %".format(round((testaccuracy*100),2)))
print("Test Data Sensitivity  :{} %".format(round((testsensitivity*100),2)))
print("Test Data Specificity  :{} %".format(round((testspecificity*100),2)))

In [ ]:
# "Support Vector Classifier"
from sklearn.svm import SVC  
clf = SVC(kernel='linear') 

In [ ]:
clf.fit(X_train, y_train) 

In [ ]:
predict_test_clf = clf.predict(X_test)
predict_test_clf

In [ ]:
testaccuracy_clf = accuracy_score(y_test, predict_test_clf)
print('accuracy_score on test dataset : ', testaccuracy_clf)

In [ ]:
#KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt 
import seaborn as sns

K = []
training = []
test = []
scores = {}
  
for k in range(2, 21):
    clf = KNeighborsClassifier(n_neighbors = k)
    clf.fit(X_train, y_train)
  
    training_score = clf.score(X_train, y_train)
    test_score = clf.score(X_test, y_test)
    K.append(k)
  
    training.append(training_score)
    test.append(test_score)
    scores[k] = [training_score, test_score]

In [ ]:
for keys, values in scores.items():
    print(keys, ':', values)

In [ ]:
ax = sns.stripplot(K, test);
ax.set(xlabel ='values of k', ylabel ='Test Score')
plt.show()

In [5]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
final_df["BusinessTravel"] = le.fit_transform(final_df["BusinessTravel"])
final_df["Department"] = le.fit_transform(final_df["Department"])
final_df["EducationField"] = le.fit_transform(final_df["EducationField"])
final_df["Gender"] = le.fit_transform(final_df["Gender"])
final_df["Attrition"] = le.fit_transform(final_df["Attrition"])
final_df["JobRole"] = le.fit_transform(final_df["JobRole"])
final_df["MaritalStatus"] = le.fit_transform(final_df["MaritalStatus"])
final_df["Over18"] = le.fit_transform(final_df["Over18"])

In [7]:
from sklearn.preprocessing import normalize 
X_normalized=normalize(final_df1,axis=0)

In [8]:
total_length=len(final_df1)
train_length=int(0.8*total_length)
test_length=int(0.2*total_length)

In [9]:
X_train=X_normalized[:train_length]
X_test=X_normalized[train_length:]
y_train=attrition_lable1[:train_length]
y_test=attrition_lable1[train_length:]

In [10]:
from keras.utils import np_utils
y_train=np_utils.to_categorical(y_train,num_classes=2)
y_test=np_utils.to_categorical(y_test,num_classes=2)
print("Shape of y_train",y_train.shape)
print("Shape of y_test",y_test.shape)

Shape of y_train (3440, 2)
Shape of y_test (860, 2)


In [11]:
#Neural network module
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense,Activation,Dropout 
from tensorflow.keras.layers import BatchNormalization 

In [15]:
model=Sequential()
model.add(Dense(1000,input_dim=28,activation='relu'))
model.add(Dense(500,activation='relu'))
model.add(Dense(300,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 1000)              29000     
_________________________________________________________________
dense_5 (Dense)              (None, 500)               500500    
_________________________________________________________________
dense_6 (Dense)              (None, 300)               150300    
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 602       
Total params: 680,402
Trainable params: 680,402
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size=20,epochs=10,verbose=1)

Epoch 1/10
172/172 [==============================] - 5s 8ms/step - loss: 0.4552 - accuracy: 0.8328 - val_loss: 0.4131 - val_accuracy: 0.8465
Epoch 2/10
172/172 [==============================] - 1s 4ms/step - loss: 0.4228 - accuracy: 0.8363 - val_loss: 0.3940 - val_accuracy: 0.8465
Epoch 3/10
172/172 [==============================] - 1s 4ms/step - loss: 0.4072 - accuracy: 0.8363 - val_loss: 0.3862 - val_accuracy: 0.8465
Epoch 4/10
172/172 [==============================] - 1s 4ms/step - loss: 0.3930 - accuracy: 0.8410 - val_loss: 0.3841 - val_accuracy: 0.8453
Epoch 5/10
172/172 [==============================] - 1s 4ms/step - loss: 0.3933 - accuracy: 0.8448 - val_loss: 0.3777 - val_accuracy: 0.8500
Epoch 6/10
172/172 [==============================] - 1s 4ms/step - loss: 0.3884 - accuracy: 0.8477 - val_loss: 0.3952 - val_accuracy: 0.8477
Epoch 7/10
172/172 [==============================] - 1s 4ms/step - loss: 0.3903 - accuracy: 0.8459 - val_loss: 0.4068 - val_accuracy: 0.8570
Epoch 

In [18]:
prediction=model.predict(X_test)
length=len(prediction)
y_label=np.argmax(y_test,axis=1)
predict_label=np.argmax(prediction,axis=1)

accuracy=np.sum(y_label==predict_label)/length * 100 
print("Accuracy of the dataset",accuracy )

Accuracy of the dataset 85.11627906976744
